# Rendering OpenAI Gym Envs on Binder and Google Colab 
> Notes on solving a mildly tedious (but important) problem

- branch: 2020-04-16-remote-rendering-gym-envs
- badges: true
- image: images/gym-colab-binder.png
- comments: true
- author: David R. Pugh
- categories: [openai, binder, google-colab]

Getting [OpenAI](https://openai.com/) [Gym](https://gym.openai.com/docs/) environments to render properly in remote environments such as [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) and [Binder](https://mybinder.org/) turned out to be more challenging than I expected. In this post I lay out my solution in the hopes that I might save others time and effort to work it out independently.

# Google Colab Preamble

If you wish to use Google Colab, then this section is for you! Otherwise, you can skip to the next section for the Binder Preamble.

## Install X11 system dependencies

Install necessary [X11](https://en.wikipedia.org/wiki/X_Window_System) dependencies, in particular [Xvfb](https://www.x.org/releases/X11R7.7/doc/man/man1/Xvfb.1.xhtml), which is an X server that can run on machines with no display hardware and no physical input devices. 

In [10]:
!apt-get install -y xvfb x11-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


## Install additional Python dependencies

Now that you have installed Xvfb, you need to install a Python wrapper 
[`pyvirtualdisplay`](https://github.com/ponty/PyVirtualDisplay) in order to interact with Xvfb 
virtual displays from within Python. Next you need to install the Python bindings for 
[OpenGL](https://www.opengl.org/): [PyOpenGL](http://pyopengl.sourceforge.net/) and 
[PyOpenGL-accelerate](https://pypi.org/project/PyOpenGL-accelerate/). The former are the actual 
Python bindings, the latter is and optional set of C (Cython) extensions providing acceleration of 
common operations for slow points in PyOpenGL 3.x.

In [11]:
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Install OpenAI Gym

Next you need to install the OpenAI Gym package. Note that depending on which Gym environment you are interested in working with you may need to add additional dependencies. Since I am going to simulate the LunarLander-v2 environment in my demo below I need to install the `box2d` extra which enables Gym environments that depend on the [Box2D](https://box2d.org/) physics simulator.

In [3]:
!pip install gym[box2d]==0.17.* 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 3.7 MB/s 
     |████████████████████████████████| 1.0 MB 35.6 MB/s 
     |████████████████████████████████| 374 kB 41.6 MB/s 
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654651 sha256=16d2b771870870b215e1cf8ed7fd9dc37df7dd3e6b8fd41d1d5da8f28de2330c
  Stored in directory: /root/.cache/pip/wheels/84/40/e7/14efb9870cfc92ac236d78cb721dce614ddec9666c8a5e0a35
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Successfully built gym
Failed to build box2d-py
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
    Running setup.py install for box2d-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-

## Create a virtual display in the background

Next you need to create a virtual display in the background which the Gym Envs can connect to for rendering purposes. You can check that there is no display at present by confirming that the value of the [`DISPLAY`](https://askubuntu.com/questions/432255/what-is-the-display-environment-variable) environment variable has not yet been set. 

In [4]:
!echo $DISPLAY

The code in the cell below creates a virtual display in the background that your Gym Envs can connect to for rendering. You can adjust the `size` of the virtual buffer as you like but you must set `visible=False` when working with Xvfb. 

**This code only needs to be run once per session to start the display.**

In [12]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

After running the cell above you can echo out the value of the `DISPLAY` environment variable again to confirm that you now have a display running.

In [13]:
!echo $DISPLAY

:1009


For convenience I have gathered the above steps into two cells that you can copy and paste into the top of you Google Colab notebooks.

In [7]:
%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Running setup.py clean for box2d-py
Failed to build box2d-py
    Running setup.py install for box2d-py: started
    Running setup.py install for box2d-py: finished with status 'error'


  ERROR: Failed building wheel for box2d-py
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-5g3t65j0/box2d-py_c29974da9bbc444d9fe213f61b1158e8/setup.py'"'"'; __file__='"'"'/tmp/pip-install-5g3t65j0/box2d-py_c29974da9bbc444d9fe213f61b1158e8/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-dojmqnkc/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.8/box2d-py Check the logs for full command output.


CalledProcessError: ignored

In [14]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

# Binder Preamble

If you wish to use Binder, then this section is for you! Although there really isn't much of anything that needs doing.

## No additional installation required!

Unlike Google Colab, with Binder you can bake all the required dependencies (including the X11 system dependencies!) into the Docker image on which the Binder instance is based using Binder config files. These config files can either live in the root directory of your Git repo or in a `binder` sub-directory as is this case here. If you are interested in learning more about Binder, then check out the documentation for [BinderHub](https://binderhub.readthedocs.io/en/latest/) which is the underlying technology behind the Binder project.

In [ ]:
# config file for system dependencies
!cat ../binder/apt.txt

freeglut3-dev
xvfb
x11-utils


In [ ]:
# config file describing the conda environment
!cat ../binder/environment.yml

name: null

channels:
  - conda-forge
  - defaults

dependencies:
  - gym-box2d=0.17
  - jupyterlab=2.0
  - matplotlib=3.2
  - pip=20.0
  - python=3.7
  - pyvirtualdisplay=0.2


In [ ]:
# config file containing python deps not avaiable via conda channels
!cat ../binder/requirements.txt

PyOpenGL==3.1.*
PyOpenGL-accelerate==3.1.*


## Create a virtual display in the background

Next you need to create a virtual display in the background which the Gym Envs can connect to for rendering purposes. You can check that there is no display at present by confirming that the value of the [`DISPLAY`](https://askubuntu.com/questions/432255/what-is-the-display-environment-variable) environment variable has not yet been set.

In [9]:
!echo $DISPLAY

:1005


The code in the cell below creates a virtual display in the background that your Gym Envs can connect to for rendering. You can adjust the `size` of the virtual buffer as you like but you must set `visible=False` when working with Xvfb. 

**This code only needs to be run once per session to start the display.**

In [ ]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_display.start()

After running the cell above you can echo out the value of the `DISPLAY` environment variable again to confirm that you now have a display running.

In [ ]:
!echo $DISPLAY

# Demo

Just to prove that the above setup works as advertised I will run a short simulation. First I will define an `Agent` that chooses an action randomly from the set of possible actions and the define a function that can be used to create such agents.

In [ ]:
import typing

import numpy as np


# represent states as arrays and actions as ints
State = np.array
Action = int

# agent is just a function! 
Agent = typing.Callable[[State], Action]


def uniform_random_policy(state: State,
                          number_actions: int,
                          random_state: np.random.RandomState) -> Action:
    """Select an action at random from the set of feasible actions."""
    feasible_actions = np.arange(number_actions)
    probs = np.ones(number_actions) / number_actions
    action = random_state.choice(feasible_actions, p=probs)
    return action


def make_random_agent(number_actions: int,
                      random_state: np.random.RandomState = None) -> Agent:
    """Factory for creating an Agent."""
    _random_state = np.random.RandomState() if random_state is None else random_state
    return lambda state: uniform_random_policy(state, number_actions, _random_state)
    

In the cell below I wrap up the code to simulate a single epsiode of an OpenAI Gym environment. Note that the implementation assumes that the provided environment supports `rgb_array` rendering (which not all Gym environments support!).

In [ ]:
import gym
import matplotlib.pyplot as plt
from IPython import display


def simulate(agent: Agent, env: gym.Env) -> None:
    state = env.reset()
    img = plt.imshow(env.render(mode='rgb_array'))
    done = False
    while not done:
        action = agent(state)
        img.set_data(env.render(mode='rgb_array')) 
        plt.axis('off')
        display.display(plt.gcf())
        display.clear_output(wait=True)
        state, reward, done, _ = env.step(action)       
    env.close()
    


Finally you can setup your desired environment...

In [ ]:
lunar_lander_v2 = gym.make('LunarLander-v2')
_ = lunar_lander_v2.seed(42)

...and run a simulation!

In [ ]:
random_agent = make_agent(lunar_lander_v2.action_space.n, random_state=None)
simulate(random_agent, lunar_lander_v2)

Currently there appears to be a non-trivial amount of flickering during the simulation. Not entirely sure what is causing this undesireable behavior. If you have any idea how to improve this, please leave a comment below. I will be sure to update this post accordingly if I find a good fix.